# Google Earth Engine Javascript code

In [ ]:
// Define the Red River Valley area of interest (AOI)
var aoi = ee.Geometry.Rectangle([-97.5, 48.5, -96.5, 49.5]);

// Define the date range for Landsat 9 imagery
var startDate = '2022-09-01';
var endDate = '2022-09-30';

// Load Landsat 9 surface reflectance data
var landsat9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
  .filterDate(startDate, endDate)
  .filterBounds(aoi);

// Define visualization parameters for Landsat 9 imagery
var visParams = {
  bands: ['SR_B4', 'SR_B3', 'SR_B2'],
  min: 0,
  max: 30000,
  gamma: 1.4
};

// Add the Red River Valley AOI to the map
Map.addLayer(aoi, {}, 'Red River Valley AOI');

// Add the Landsat 9 imagery to the map
Map.addLayer(landsat9.median().clip(aoi), visParams, 'Landsat 9 September 2022');

// Center the map on the Red River Valley AOI
Map.centerObject(aoi, 9);


# Imagery Classification Javascript code

In [ ]:
// Define the bands to use for classification
var bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'];

// Load the training points as a FeatureCollection
var trainingPoints = ee.FeatureCollection('wang1/wangmaochuan/training_points');

// Sample the image at the training points
var trainingData = landsat9.select(bands).sampleRegions({
  collection: trainingPoints,
  properties: ['class'],
  scale: 30
});

// Train a classifier
var classifier = ee.Classifier.smileRandomForest(10).train({
  features: trainingData,
  classProperty: 'class',
  inputProperties: bands
});

// Perform the classification
var classifiedImage = landsat9.select(bands).classify(classifier);

// Define visualization parameters for classified image
var classVisParams = {
  min: 1,
  max: 6,
  palette: ['red', 'green', 'blue', 'yellow', 'orange', 'purple']
};

// Add the AOI, Landsat 9 imagery, and classified image to the map
Map.addLayer(aoi, {}, 'Red River Valley AOI');
Map.addLayer(landsat9, {bands: ['SR_B4', 'SR_B3', 'SR_B2'], min: 0, max: 30000, gamma: 1.4}, 'Landsat 9 September 2022');
Map.addLayer(classifiedImage, classVisParams, 'Classified Image');

// Center the map on the Red River Valley AOI
Map.centerObject(aoi, 9);


# flood modeling 


In [ ]:
import arcpy
from arcpy import env
from arcpy.sa import *

# Set environment settings
env.workspace = r"C:\Users\wang1\Documents\ArcGIS\Projects\MyProject35\MyProject35.gdb"  
elevation_dem = r"C:\Users\wang1\Documents\ArcGIS\Projects\MyProject35\elevation_dem.tif"  

# Check out the Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")

# Fill sinks in the DEM to remove small imperfections
filled_dem = Fill(elevation_dem, 0.1)  # Adjust the z-limit parameter as needed 
filled_dem.save(r"C:\Users\wang1\Documents\ArcGIS\Projects\MyPeoject35\feilled_den.tif")

# Calculate the flow direction
flow_direction = FlowDirection(filled_dem)
flow_direction.save(r"C:\Users\Wang1\documents\ArcGIS\Projects\Myprojects35\flow_direction.tif")

# Calculate the flow accumulation
flow_accumulation = FlowAccumulation(flow_direction)
flow_accumulation.save(r"C:\Users\Wang1\documents\ArcGIS\Projects\Myprojects35\flow_accumulation.tif")

# Define the stream network
stream_network = Con(flow_accumulation, 1, "", "VALUE > 100")  # Adjust the threshold value as needed (100 in this example)
stream_network.save(r"C:\Users\Wang1\documents\ArcGIS\Projects\Myprojects35\stream_network.tif")

# Convert the stream network raster to a polyline feature
StreamToFeature(stream_network, flow_direction, "stream_network.shp", "NO_SIMPLIFY")

# Calculate the line density of the stream network
line_density = LineDensity("stream_network.shp", None, 30)  # Adjust the search radius as needed (30 in this example)
line_density.save("line_density.tif")

# Calculate the Euclidean distance to the stream network
euclidean_distance = EucDistance("stream_network.shp")
euclidean_distance.save("C:\Users\Wang1\documents\ArcGIS\Projects\Myprojects35\euclidean_distance.tif")

# Check in the Spatial Analyst extension
arcpy.CheckInExtension("Spatial")




# Weighted Overlay

In [ ]:
# Input raster files
elevation = r"C:\Users\wang1\Documents\ArcGIS\Projects\MyProject35\elevation_dem.tif"
drainage_density = r"C:\Users\wang1\Documents\ArcGIS\Projects\MyProject35\drainage_density.tif"
slope_raster = r"C:\Users\wang1\Documents\ArcGIS\Projects\MyProject35\slope.tif"
distance_to_streams = r"C:\Users\wang1\Documents\ArcGIS\Projects\MyProject35\distance_to_streams.tif"
land_cover = r"C:\Users\wang1\Documents\ArcGIS\Projects\MyProject35\land_cover.tif"

# Normalize input rasters using Min-Max normalization
def min_max_norm(raster):
    min_val = arcpy.GetRasterProperties_management(raster, "MINIMUM")
    max_val = arcpy.GetRasterProperties_management(raster, "MAXIMUM")
    return (Raster(raster) - float(min_val.getOutput(0))) / (float(max_val.getOutput(0)) - float(min_val.getOutput(0)))

normalized_elevation = min_max_norm(elevation)
normalized_drainage_density = min_max_norm(drainage_density)
normalized_slope = min_max_norm(slope_raster)
normalized_distance_to_streams = min_max_norm(distance_to_streams)
normalized_land_cover = min_max_norm(land_cover)

# Define weights
weights = {
    "elevation": 0.3,
    "drainage_density": 0.2,
    "slope": 0.05,
    "distance_to_streams": 0.1,
    "land_cover": 0.35
}

# Perform the weighted overlay
flood_model = (
    normalized_elevation * weights["elevation"] +
    normalized_drainage_density * weights["drainage_density"] +
    normalized_slope * weights["slope"] +
    normalized_distance_to_streams * weights["distance_to_streams"] +
    normalized_land_cover * weights["land_cover"]
)

# Save the output raster
flood_model.save(r"C:\Users\wang1\Documents\ArcGIS\Projects\MyProject35\flood_model.tif")

# Generate Tessellation

In [ ]:
# Additional imports
from arcpy import mp

# Set the tessellation parameters
tessellation_type = "HEXAGON"  # Choose between "HEXAGON", "SQUARE", or "TRIANGLE"
tessellation_size = 1000  # Set the size of the tessellation polygons

# Define the tessellation output feature class
tessellation_output = r"C:\Users\wang1\Documents\ArcGIS\Projects\MyProject35\flood_model_tessellation.shp"

# Create a bounding box for the tessellation based on the flood model extent
flood_model_extent = arcpy.Describe(r"C:\Users\wang1\Documents\ArcGIS\Projects\MyProject35\flood_model.tif").extent
bounding_box = arcpy.Polygon(arcpy.Array([flood_model_extent.lowerLeft,
                                          flood_model_extent.lowerRight,
                                          flood_model_extent.upperRight,
                                          flood_model_extent.upperLeft,
                                          flood_model_extent.lowerLeft]))

# Generate the tessellation
arcpy.GenerateTessellation_management(tessellation_output, bounding_box, tessellation_type, tessellation_size)

# Create a new map in the current project
aprx = mp.ArcGISProject("CURRENT")
map_obj = aprx.maps[0]

# Add the flood model tessellation to the map
tessellation_layer = map_obj.addDataFromPath(env.workspace + "/" + tessellation_output)
tessellation_layer.name = "Flood Model Tessellation